# Projet 4 Bernadoy Corentin

Le but de ce projet est de créer une balise html `<a>` autour des textes de référencés dans les différents arrêtés municipaux mis à notre disposition.

## Imports

In [1]:
import os
import bs4
import regex as re

## Stratégie et étude du jeu de donnée

Pour une première approche, nous allons tenter d'utiliser un maximum les expressions régulières. En effet, il s'agit d'une approche simple et si elle réussit, permet de consitituer un premier jeu de données labélisées pour une étude par machine learning. 

In [55]:
regex_list = [
    r"directive[\sA-Za-zÀ-ÿ]+\d{4}/\d+/[a-zA-Z]{2}",
]

In [54]:
with open('data/data/0005302750/2011-01-13_APC-auto_pixtral.html', 'r') as f:
    soup = bs4.BeautifulSoup(f, 'html')   
    whole_text = soup.find_all(['div', 'td', 'h2'])
    for tag in whole_text:
        if (find_regex(regex_list, tag.text)[0]):
            print(tag.text)


    VU la directive européenne 2008/105/EC du 16 décembre 2008 établissant des normes de qualité environnementale dans le domaine de l'eau ;
   

    VU la directive européenne 2006/11/CE concernant la pollution causée par certaines substances dangereuses déversées dans le milieu aquatique de la Communauté ;
   

    VU la directive européenne 2000/60/CE du 23 octobre 2000 établissant un cadre pour une politique communautaire dans le domaine de l'eau (DCE) ;
   

    Considérant l'objectif de respect des normes de qualité environnementale dans le milieu en 2015 fixé par la directive européenne 2000/60/CE ;
   



       Substances Dangereuses Prioritaires issues de l’annexe X de la DCE (tableau A de la circulaire du 07/05/07) et de la directive filière DCE adoptée le 20 octobre 2008 (anthracène et endosulfan)
      

       Substances Prioritaires issues de l’annexe X de la DCE (tableau A de la circulaire du 07/05/07)
      

       Autres substances pertinentes issues de la liste 1 d

In [56]:
def find_regex(regex_list, text):
    occurences = []
    for regex in regex_list:
        for occurence in re.finditer(regex, text):
            occurences.append((occurence.start(0),occurence.end(0)))
    return((len(occurences) > 0), occurences)

In [57]:
def rewriting(soup, text, occurences):
    original_text = text.string
    for occ in occurences:

        prefix = original_text[:occ[0]]
        wrapped = original_text[occ[0]:occ[1]]
        suffix = original_text[occ[1]:]

        text.string = ''

        text.append(prefix)

        a_tag = soup.new_tag('a')
        a_tag.string = wrapped
        text.append(a_tag)

        text.append(suffix)

        original_text = text.string

In [58]:
#cellule pour créer les dossiers de résultats
root = "./data/"
os.makedirs(root + "results/", exist_ok= True)
sub_data_dirs = next(os.walk(root + 'data/'))[1]
for sub_data_dir in sub_data_dirs:
    os.makedirs(root + "results/" + sub_data_dir, exist_ok= True)

In [59]:
root = "./data/"
def file_searcher(root = root):
    data_dirs = root + 'data/'
    sub_data_dirs = next(os.walk(data_dirs))[1]
    res_dirs = root + 'results/'
    for sub_data_dir in sub_data_dirs:
        file_names = next(os.walk(data_dirs + sub_data_dir + '/'))[2]
        for file_name in file_names:
            data_path = data_dirs + sub_data_dir + '/' + file_name
            res_data_path = res_dirs + sub_data_dir + '/' + file_name[:-5] + '_results.html'
            with open(data_path, 'r') as f:
                soup = bs4.BeautifulSoup(f, 'html')   
                whole_text = soup.find_all(['div', 'td', 'h2'])
                for tags in whole_text:
                    found, occurences = find_regex(regex_list, tags.text)
                    if found:
                        rewriting(soup, tags, occurences)

            with open(res_data_path, 'w', encoding= "utf-8") as f:
                    f.write(str(soup))

In [60]:
root = "./data/"
data_dirs = root + 'data/'
sub_data_dirs = next(os.walk(data_dirs))[1]
res_dirs = root + 'results/'
for sub_data_dir in sub_data_dirs:
    file_names = next(os.walk(data_dirs + sub_data_dir + '/'))[2]
    for file_name in file_names:
        data_path = data_dirs + sub_data_dir + '/' + file_name
        res_data_path = res_dirs + sub_data_dir + '/' + file_name[:-5] + '_results.html'
        with open(data_path, 'r') as f:
            soup = bs4.BeautifulSoup(f, 'html')   
            whole_text = soup.find_all(['div', 'td', 'h2'])
            for tag in whole_text:
                if((tag.text.find('directive') != -1)):
                    print(tag.text)


    VU la directive 2010/75/UE du 24 novembre 2010 relative aux émissions industrielles, dite à directive IED) et les conclusions sur les meilleures techniques disponibles relatives, notamment, à l'incinération de déchets (BREF et WtE) ;
   

    VU la directive 2003/87/CE du 13 octobre 2003 établissant un système d'échange de quotas d'émission de gaz à effet de serre dans la Communauté et modifiant la directive 96/61/CE du Conseil ;
   

       L’établissement possède des installations visées par la directive 2010/75/UE du 24 novembre 2010 relative aux émissions industrielles, dite « directive IED ». Ces installations relèvent de la rubrique n° 3520-a de la nomenclature des installations classées et sont soumises aux dispositions des articles R. 515-60 à R. 515-84 du code de l’environnement.
      

       Les appareils et systèmes de protection destinés à être utilisés dans les emplacements où des atmosphères explosives peuvent se présenter doivent être sélectionnés conformément aux

In [61]:
file_searcher()

TypeError: 'NoneType' object is not subscriptable